# Supplementary Figure 1: Phase Portrait of Thiele Equation Solutions for Zero Damping

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import patches
from scipy.integrate import solve_ivp

import sys
sys.path.append('../..')
from Utils.PhaseDiagramFunctions import *

plt.style.use('Style.mplstyle')

# Directory in which collective coordinate integration data is stored
cc_directory = '../../CollectiveCoordinates/CollectiveCoordinateIntegration/CollectiveCoordinateDataSupplied'

# Obtain parameters such as magnetic field, fit parameters (see Methods section of manuscript on fitting)
# E_exchange, E_magnetic, and E_electric are the contributions from exchange, Zeeman, and electric field to energy respectively
# alpha is the Gilbert damping constant
# F_rex is -dU_ex / dR
# Gamma11 and Gamma22 are as in the manuscript where 1 is R and 2 is η
# G12 is the G_{Rη} from the manuscript

# What I call Xi are parameters that show up in the analytical expressions for the helicity-dependent
# parts of the generalised forces for the skyrmion profile ansatz we use

# Xi1 is the integral of cos(2θ) dθ / dR over ρ
# Xi2 is the integral of ρ * d^2θ / dRdρ over ρ
# Xi3 is the integral of cos(θ)sin(θ) over ρ
# Xi4 is the integral of ρ * dθ / dρ over ρ
Bz, dw_width, E_exchangeFitParams, E_magnetic_integralFitParams, E_electric_integralFitParams, alpha, F_RexFitParams, Gamma11FitParams, Gamma22FitParams, G12FitParams, Xi1FitParams, Xi2FitParams, Xi3FitParams, Xi4FitParams, optimalSkyrmionRadius = read_parameters(cc_directory)


def time_derivatives(t, y):
    
    """
    Calculate d/dt(R) and d_dt(eta) for the numerical integration.
    """

    R, eta = y

    Ez = 0.     # Electric field
    alpha = 0.  # Gilbert damping constant

    # G12 is the G_{Rη} from the manuscript
    # Gamma11 and Gamma22 are as in the manuscript where 1 is R and 2 is η
    # F_rex is -dU_ex / dR
    # F_R and F_eta are -dU/dR and -dU/dη respectively

    # What I call Xi are parameters that show up in the analytical expressions for the helicity-dependent
    # parts of the generalised forces for the skyrmion profile ansatz we use
    
    # Xi1 is the integral of cos(2θ) dθ / dR over ρ
    # Xi2 is the integral of ρ * d^2θ / dRdρ over ρ
    # Xi3 is the integral of cos(θ)sin(θ) over ρ
    # Xi4 is the integral of ρ * dθ / dρ over ρ

    Gamma11 = alpha * inverse_linear_fit(R, Gamma11FitParams[0], Gamma11FitParams[1], Gamma11FitParams[2], Gamma11FitParams[3])
    Gamma22 = alpha * linear_fit(R, Gamma22FitParams[0], Gamma22FitParams[1])
    G12 = linear_fit(R, G12FitParams[0], G12FitParams[1])
    F_Rex = inverse_fourth_order_fit(R, F_RexFitParams[0], F_RexFitParams[1], F_RexFitParams[2], F_RexFitParams[3], F_RexFitParams[4], F_RexFitParams[5])
    Xi1 = inverse_fourth_order_fit(R, Xi1FitParams[0], Xi1FitParams[1], Xi1FitParams[2], Xi1FitParams[3], Xi1FitParams[4], Xi1FitParams[5])
    Xi2 = inverse_fourth_order_fit(R, Xi2FitParams[0], Xi2FitParams[1], Xi2FitParams[2], Xi2FitParams[3], Xi2FitParams[4], Xi2FitParams[5])
    Xi3 = inverse_fourth_order_fit(R, Xi3FitParams[0], Xi3FitParams[1], Xi3FitParams[2], Xi3FitParams[3], Xi3FitParams[4], Xi3FitParams[5])
    Xi4 = linear_fit(R, Xi4FitParams[0], Xi4FitParams[1])
    
    F_R = F_Rex - Bz*G12 + Ez*np.cos(eta)*(Xi1 + Xi2)
    F_eta = -Ez*np.sin(eta)*(Xi3 + Xi4)

    # The prefactor that goes before both \dot{R} and \dot{\eta}
    prefactor = 1 / (G12**2 + Gamma11*Gamma22)

    # Time derivatives of R and η to be output
    Rdot = prefactor * (Gamma22*F_R + G12*F_eta)
    etadot = prefactor * (Gamma11*F_eta - G12*F_R)
    
    return np.array([Rdot, etadot])


def plot_R_eta(ax, solutions_arrays=[]):
    
    """
    Bottom plots of radius and helicity over time.
    """

    # Points in the R-eta plane to plot flow lines
    Rmesh = np.linspace(1., 2., 31)
    etamesh = np.linspace(0, 2*np.pi, 31)
    RMESH, ETAMESH = np.meshgrid(Rmesh, etamesh)

    # Initial values of dR/dt and dη/dt
    dR_dt_init, deta_dt_init = time_derivatives(0, [RMESH, ETAMESH]) 
    magnitude = np.sqrt(dR_dt_init**2 + deta_dt_init**2)

    # The x-values for which the flowlines should be plotted
    x_positions = np.linspace(Rmesh.min(), Rmesh.max(), 20)

    for i in range(len(x_positions)):
        
        x_index = np.argmin(np.abs(x_positions[i] - Rmesh))
        this_magnitude = magnitude[magnitude.shape[0]//2, x_index]
        start_y_pos = 1.0
        end_y_pos = 1.03 if deta_dt_init[magnitude.shape[0]//2, x_index] > 0 else 0.97

        ax.axvline(x_positions[i], linewidth=7*this_magnitude/magnitude.max(), color='gray')
        arrow = patches.FancyArrowPatch(posA=(x_positions[i], start_y_pos), posB=(x_positions[i], end_y_pos), 
                                        arrowstyle='-|>,head_length=7, head_width=5', linewidth=0, fill=True, color='gray')
        ax.add_patch(arrow)

    # The skyrmion radius which minimises its energy
    minR = optimalSkyrmionRadius[0]
    ax.axvline(minR, linestyle='--', color='black')
    ax.text(minR + 0.01, 1.85, r'$R^*$', fontsize=16)

    ax.set_xlabel(r'$R$')
    ax.set_ylabel(r'$\eta / \pi$')
    ax.set_xlim(np.min(Rmesh), np.max(Rmesh))
    ax.set_ylim(np.min(etamesh/np.pi), np.max(etamesh/np.pi))

    for array in solutions_arrays:
        ax.plot(array[0], array[1] % 2*np.pi, linewidth=3)


if __name__ == '__main__':

    fig = plt.figure(figsize=(8, 6))
    gs = GridSpec(3, 2, figure=fig, wspace=0.4, hspace=1)

    ax1 = fig.add_subplot(gs[:2, :])
    ax2 = fig.add_subplot(gs[2, 0])
    ax3 = fig.add_subplot(gs[2, 1])

    times = np.linspace(0, 10, 100)

    solutions = []

    minR = optimalSkyrmionRadius[0]
    distance_from_Rstar = 0.25  # Distance on either side of optimal radius for coloured lines
    initial_R_values = [minR - distance_from_Rstar, minR + distance_from_Rstar]

    eta_axis_lim = 0

    for i in range(len(initial_R_values)):

        # Time integration for bottom plots
        initial_values = [initial_R_values[i], 0]
        sol = solve_ivp(time_derivatives, [np.min(times), np.max(times)], initial_values, t_eval=times, method='Radau')
        solutions.append([sol.y[0, :], sol.y[1, :]])

        ax2.plot(times, sol.y[0, :])
        ax3.plot(times, sol.y[1, :] / np.pi)

        # Symmetric axis extent about eta=0 in eta-t plot
        new_eta_axis_lim = np.max(np.abs(sol.y[1, :] / np.pi))
        if new_eta_axis_lim > eta_axis_lim: eta_axis_lim = new_eta_axis_lim

    plot_R_eta(ax1, solutions)

    ax2.set_xlabel(r'$t$')
    ax2.set_ylabel(r'$R$')

    ax2.axhline(minR, linestyle='--', color='black')
    ax2.text(10.7, minR + 0.01, r'$R^*$', fontsize=16)

    ax2.set_ylim(1, 2)

    ax3.set_xlabel(r'$t$')
    ax3.set_ylabel(r'$\eta / \pi$')

    ax3.set_ylim(-1.1*eta_axis_lim, 1.1*eta_axis_lim)